In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
DATA_DIR = 'datasets/ml-data/'

In [3]:
df = pd.read_csv(DATA_DIR + 'ratings.csv')

In [4]:
df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config('spark.driver.memory', '8G')
    .config('spark.sql.analyzer.failAmbiguousSelfJoin', 'False')
    .master("local[*]")
    .getOrCreate()
)

In [6]:
# считываем данные из CSV
# и преобразуем времяпроставления оценки из целого числа в дату мл временем
import os
import pyspark.sql.functions as sql_func

ratings = (
    spark
    .read
    .csv(
        os.path.join(DATA_DIR, 'ratings.csv'),
        header=True,
        inferSchema=True
    )
    # если используется меньше памяти,
    # то здесь можно взять не все данные, а небольшую выборку
    # даже при fraction=.01 качественная картина не меняеся
    .sample(withReplacement=False, fraction=1.0, seed=0)
    .withColumn('rating_datetime', sql_func.from_unixtime('timestamp'))
    .drop('timestamp')
    .cache()
)

In [7]:
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|    rating_datetime|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-28 00:00:21|
|     1|    481|   3.5|2009-10-28 00:04:16|
|     1|   1091|   1.5|2009-10-28 00:04:31|
|     1|   1257|   4.5|2009-10-28 00:04:20|
|     1|   1449|   4.5|2009-10-28 00:01:04|
|     1|   1590|   2.5|2009-10-28 00:00:36|
|     1|   1591|   1.5|2009-10-28 00:04:35|
|     1|   2134|   4.5|2009-10-28 00:04:24|
|     1|   2478|   4.0|2009-10-28 00:00:39|
|     1|   2840|   3.0|2009-10-28 00:05:00|
|     1|   2986|   2.5|2009-10-28 00:04:56|
|     1|   3020|   4.0|2009-10-28 00:01:00|
|     1|   3424|   4.5|2009-10-28 00:04:04|
|     1|   3698|   3.5|2009-10-28 00:00:43|
|     1|   3826|   2.0|2009-10-28 00:00:10|
|     1|   3893|   3.5|2009-10-28 00:04:46|
|     2|    170|   3.5|2007-10-21 00:53:01|
|     2|    849|   3.5|2007-10-21 00:52:17|
|     2|   1186|   3.5|2007-10-21 00:53:31|
|     2|   1235|   3.0|2007-10-2

In [8]:
# оцениваем размеры данных
print('всего пользователей', ratings.select('userId').distinct().count())
print('всего фильмов', ratings.select('movieId').distinct().count())
print('всего оценок', ratings.count())

всего пользователей 283228
всего фильмов 53889
всего оценок 27753444


In [23]:
print(ratings.select('*').show())

+------+-------+------+-------------------+
|userId|movieId|rating|    rating_datetime|
+------+-------+------+-------------------+
|     1|    307|   3.5|2009-10-28 00:00:21|
|     1|    481|   3.5|2009-10-28 00:04:16|
|     1|   1091|   1.5|2009-10-28 00:04:31|
|     1|   1257|   4.5|2009-10-28 00:04:20|
|     1|   1449|   4.5|2009-10-28 00:01:04|
|     1|   1590|   2.5|2009-10-28 00:00:36|
|     1|   1591|   1.5|2009-10-28 00:04:35|
|     1|   2134|   4.5|2009-10-28 00:04:24|
|     1|   2478|   4.0|2009-10-28 00:00:39|
|     1|   2840|   3.0|2009-10-28 00:05:00|
|     1|   2986|   2.5|2009-10-28 00:04:56|
|     1|   3020|   4.0|2009-10-28 00:01:00|
|     1|   3424|   4.5|2009-10-28 00:04:04|
|     1|   3698|   3.5|2009-10-28 00:00:43|
|     1|   3826|   2.0|2009-10-28 00:00:10|
|     1|   3893|   3.5|2009-10-28 00:04:46|
|     2|    170|   3.5|2007-10-21 00:53:01|
|     2|    849|   3.5|2007-10-21 00:52:17|
|     2|   1186|   3.5|2007-10-21 00:53:31|
|     2|   1235|   3.0|2007-10-2

In [26]:
data = (
    ratings
    .select(
        'userId',
        sql_func.year('rating_datetime').alias('year'),
        sql_func.month('rating_datetime').alias('month')
    )
    .groupBy('year', 'month')
    .agg(sql_func.countDistinct('userId'))
    .orderBy('year', 'month')
    .toPandas()
)

In [27]:
data

,year,month,count(userId)
0,1995,1,2
1,1996,1,8
2,1996,2,36
3,1996,3,347
4,1996,4,1755
...,...,...,...
268,2018,5,3191
269,2018,6,3113
270,2018,7,3285
271,2018,8,3257
